<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

## 1. 🚀 Introducción y Contexto

### 🛠️ La Historia que Motivó este Análisis

Era un día cualquiera en **Tandil** cuando mi bordeadora STIHL FSE60 decidió rendirse. Necesitaba un repuesto específico y, como cualquier persona del siglo XXI, mi primer instinto fue buscarlo **online**.

🔍 **Google:** Nada en la ciudad  
🛒 **MercadoLibre:** ¡Disponible! Pero a **350 km** y con 1 día de espera

Decidí intentar la búsqueda física. Después de **un día completo** recorriendo comercios...

💡 **¡EUREKA!** Un comercio finalmente sabía dónde conseguirlo. Estaba a **10 cuadras de casa**.

#### 🤔 La Pregunta que Cambió Todo

> **"Si este comercio lo tenía, ¿por qué no apareció en mis búsquedas online?"**

---

### 🎯 Objetivos del Análisis

**🔍 Objetivo Principal:** Analizar el estado de la digitalización comercial en 5 ciudades del centro de la Provincia de Buenos Aires.

**📊 Preguntas de Investigación:**
- ¿Qué porcentaje de comercios tiene **datos completos** en Google Maps?
- ¿Cuántos tienen **sitio web** o **redes sociales** registradas?
- ¿El **tamaño de la ciudad** influye en la digitalización?

**💡 Meta:** Transformar una experiencia personal en insights útiles para comerciantes locales.

</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

## 2. 🔧 Preparación de Datos

📊 Importación y Limpieza

**🔄 Procesamiento:** Carga de múltiples archivos CSV por ciudad, selección de columnas relevantes y corrección de valores erróneos.

**🔗 Consolidación:** Concatenación de datasets individuales en archivo único 'Regional' con estandarización de teléfonos, tratamiento de missing values y eliminación de duplicados.

**🧹 Normalización:** Limpieza de nombres comerciales (comillas, asteriscos, emoticones) y estandarización de formato.

</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

### 🛠️ Tratamiento de Valores Nulos como Datos Informativos

**Decisión:** Reemplazar valores nulos con "Incomplete" en lugar de eliminarlos

**Justificación:** Los datos "faltantes" son en realidad información valiosa sobre la digitalización comercial:
- Un comercio sin website = información sobre su nivel de digitalización
- Un comercio sin teléfono registrado = dato sobre completitud de su presencia online
- Estos "vacíos" son parte del análisis: miden la brecha digital

**Impacto en el análisis:** Nos permite calcular porcentajes de comercios con:
- Presencia web completa vs incompleta
- Información de contacto disponible vs no disponible
- Nivel de digitalización por ciudad
</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

### 🛠️ Normalización de Datos de Ubicación

**Problema identificado:** Google Maps devuelve inconsistencias:
- Códigos postales en lugar de nombres de ciudad
- "Buenos Aires" genérico para comercios locales
- Abreviaciones no estándar (CJV, GEV, etc.)

**Solución:** Corrección manual basada en:
- Conocimiento local de las ciudades del estudio

</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

### 🛠️ Criterios para Identificación de Duplicados

**Campos utilizados:** title, city, street, location/lat, location/lng

**Lógica:** Un negocio es el mismo si tiene:
- Mismo nombre comercial
- Misma ubicación física (calle + coordenadas)
- Misma ciudad

**Casos especiales considerados:**
- Franquicias con mismo nombre pero distintas ubicaciones = NO duplicados
- Mismo comercio con pequeñas variaciones de nombre = SÍ duplicados
</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

### 💡 Contexto: El Problema de la Visibilidad Digital

**Situación real:** Un comercio local tenía el repuesto que necesitaba, pero no aparecía en búsquedas online.

**Pregunta de investigación:** ¿Qué tan común es este problema en ciudades del interior?

**Métricas que buscaremos:**
- % de comercios sin presencia web
- % con información de contacto incompleta
- Diferencias entre ciudades grandes vs pequeñas

</div>

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

### 💡 Limitaciones y Supuestos del Estudio

**Limitaciones de los datos:**
- Solo incluye comercios registrados en Google Maps
- Sesgado hacia comercios con algún nivel de presencia digital
- No incluye comercios que operan únicamente offline

**Supuestos realizados:**
- Google Maps es representativo de la presencia digital local
- Las 5 ciudades seleccionadas son representativas del interior bonaerense
- Los datos de scraping reflejan el estado actual (no histórico)

</div>

In [64]:
# Se importan las librerías para la carga y manipulación de los datos.
# pandas: manejo y limpieza de datos
# numpy: operaciones numéricas
# glob, os: manipulación de archivos y carpetas

import pandas as pd
import os
import numpy as np
from glob import glob
import glob

# Configura pandas para mostrar todas las columnas del DataFrame, sin recorte visual.
pd.set_option('display.max_columns', None) 


In [65]:
# Búsqueda de los archivos .csv en el directorio
glob.glob('../Datasets/*azul*.csv')

['../Datasets/azul_dataset_google-maps-extractor_2025-05-19_00-03-31-957.csv',
 '../Datasets/azul_dataset_crawler-google-places_2025-07-30_14-54-37-400.csv',
 '../Datasets/azul_dataset_crawler-google-places_2025-06-15_22-59-32-816.csv']

In [66]:
# Lectura de todos los archivos CSV de la carpeta y se combinan en un único DataFrame con 'pd.concat()' 
# y se reestablece el índice con 'reset_index(drop=True)'.
# Se listan las columnas del DataFrame resultante.
df = pd.concat(map(pd.read_csv, glob.glob('../Datasets/*azul*.csv'))).reset_index(drop=True)
df.columns

Index(['additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas',
       'additionalInfo/Ambiente/0/A la moda',
       'additionalInfo/Ambiente/1/Agradable',
       'additionalInfo/Ambiente/2/Informal',
       'additionalInfo/Ambiente/3/Relajado',
       ...
       'phonesUncertain/54', 'pinterests/0', 'reviews', 'tiktoks/0',
       'twitters/0', 'youtubes/0', 'youtubes/1', 'youtubes/2', 'youtubes/3',
       'youtubes/4'],
      dtype='object', length=555)

In [67]:
# Selección de columnas relevantes
df = df[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df.head()

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,La Casa de los Trofeos,Comercio,NaN,Rivadavia 496,NaN,Azul,Provincia de Buenos Aires,-36.774661,-59.862623,NaN,1,NaN,NaN,0,2025-05-19T00:03:00.547Z,Comercio
1,La Escoba Loca,Tienda de artículos para el hogar,NaN,Colón Nte. 117,+54 2281 50-2996,Azul,Provincia de Buenos Aires,-36.767281,-59.851848,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,44.0,73,2025-05-19T00:03:02.157Z,Comercio
2,Artículos magali,Tienda general,NaN,Monseñor Caneva Nte. 160,+54 2281 31-0611,Azul,Provincia de Buenos Aires,-36.774353,-59.841898,4.3,2,NaN,NaN,12,2025-05-19T00:03:02.157Z,Comercio
3,Autoservicio San Jorge,Comercio,NaN,Burgos Nte. 100,+54 2281 42-3119,Azul,Provincia de Buenos Aires,-36.768145,-59.851305,4.6,3,https://lh3.googleusercontent.com/p/AF1QipPV0V...,6.0,127,2025-05-19T00:03:02.157Z,Comercio
4,Asociacion Latina de Exportacion SA,Comercio,NaN,Julio Argentino Roca 846,+54 2281 43-0338,Azul,Provincia de Buenos Aires,-36.785067,-59.859254,3.3,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,4,2025-05-19T00:03:02.158Z,Comercio


In [68]:
# Información del DataFrame seleccionado a través del método .info() : 
# Número de filas y columnas
# Nombres y tipos de datos de las columnas
# Valores no nulos en cada columna
# Uso de memoria
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1730 non-null   object 
 1   categoryName  1699 non-null   object 
 2   website       377 non-null    object 
 3   street        1584 non-null   object 
 4   phone         1107 non-null   object 
 5   city          1716 non-null   object 
 6   state         1717 non-null   object 
 7   location/lat  1730 non-null   float64
 8   location/lng  1730 non-null   float64
 9   totalScore    1278 non-null   float64
 10  rank          1730 non-null   int64  
 11  imageUrl      1678 non-null   object 
 12  imagesCount   1676 non-null   float64
 13  reviewsCount  1730 non-null   int64  
 14  scrapedAt     1730 non-null   object 
 15  searchString  1730 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 216.4+ KB


In [69]:
# Rellenar valores faltantes en la columna 'city' con valor 'Azul'
df['city'] = df['city'].fillna('Azul')

In [70]:
# Cuenta cuántas veces aparece cada registro de ciudad.
df['city'].value_counts()

city
Azul            1698
Buenos Aires      11
CJV                3
GEV                3
B7300GEU           3
FRU                2
CJU                2
FAD                2
CKA                2
FVW                1
CKE                1
ILF                1
Caleufú            1
Name: count, dtype: int64

In [71]:
# Se reemplazan los datos de la columna 'city' que no son correctos
df['city'] = df['city'].replace({
    'Buenos Aires': 'Azul',
    'FRU' : 'Azul',
    'FAD' : 'Azul',
    'B7300GEU' : 'Azul',
    'CJV' : 'Azul',
    'GEV' : 'Azul',
    'CJU' : 'Azul',
    'FVW' : 'Azul',
    'CKE' : 'Azul',
    'ILF' : 'Azul',
    'CKA' : 'Azul',
    'Caleufú' : 'Azul'
})

In [72]:
# Se guarda el archivo principal de la ciudad
df.to_csv('../CleanData/AzulMain.csv', index=False)

In [73]:
# Búsqueda de los archivos .csv en el directorio
glob.glob('../Datasets/*juarez*.csv')

['../Datasets/juarez_dataset_crawler-google-places_2025-06-15_23-27-55-877.csv',
 '../Datasets/juarez_dataset_crawler-google-places_2025-07-30_15-23-16-317.csv',
 '../Datasets/juarez_dataset_crawler-google-places_2025-05-29_00-54-32-651.csv']

In [74]:
# Se carga y concatenan todos los archivos CSV que contengan 'juarez' en su nombre y reestablece el índice del DataFrame resultante.
df = pd.concat(map(pd.read_csv, glob.glob('../Datasets/*juarez*.csv'))).reset_index(drop=True)

In [75]:
df = df[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df.head()

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Agronomia y Vivero FANTINI,Ingeniero agrónomo,http://www.fantinihnos.com.ar/,"Ruta 86 y, Av. Constitución",+54 2281 40-8509,Benito Juárez,Provincia de Buenos Aires,-37.678079,-59.820755,5.0,13,https://lh3.googleusercontent.com/p/AF1QipOVmH...,3.0,2,2025-06-15T23:27:13.052Z,maquinaria
1,Lo de rosa,Kiosco,NaN,Palacio 164,NaN,Benito Juárez,Provincia de Buenos Aires,-37.675689,-59.794287,NaN,40,https://lh3.googleusercontent.com/p/AF1QipPaXl...,8.0,0,2025-06-15T23:27:17.780Z,tienda
2,Distribuidora MAGICA,Tienda de alimentación,NaN,NaN,NaN,Benito Juárez,Provincia de Buenos Aires,-37.677326,-59.804705,NaN,39,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T23:27:17.780Z,tienda
3,Diaz Martin J,Carnicería,NaN,Chacabuco 290,+54 2292 45-3300,Benito Juárez,Provincia de Buenos Aires,-37.676439,-59.798500,4.0,60,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,1,2025-06-15T23:27:19.011Z,tienda
4,Elegante sport,Tienda de ropa de deportes,https://www.instagram.com/elegante_sport_bj?ig...,Ameghino 25,+54 2281 30-1465,Benito Juárez,Provincia de Buenos Aires,-37.681722,-59.815709,NaN,59,https://lh3.googleusercontent.com/p/AF1QipPPZN...,2.0,0,2025-06-15T23:27:19.011Z,tienda


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         704 non-null    object 
 1   categoryName  686 non-null    object 
 2   website       125 non-null    object 
 3   street        655 non-null    object 
 4   phone         430 non-null    object 
 5   city          701 non-null    object 
 6   state         701 non-null    object 
 7   location/lat  704 non-null    float64
 8   location/lng  704 non-null    float64
 9   totalScore    451 non-null    float64
 10  rank          704 non-null    int64  
 11  imageUrl      681 non-null    object 
 12  imagesCount   681 non-null    float64
 13  reviewsCount  704 non-null    int64  
 14  scrapedAt     704 non-null    object 
 15  searchString  704 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 88.1+ KB


In [77]:
# Rellenar valores faltantes en la columna 'city'
df['city'] = df['city'].fillna('Benito Juárez')

In [78]:
# Se cuenta la frecuencia de los registros de la columna 'city'
df['city'].value_counts()

city
Benito Juárez    690
Buenos Aires       6
Henderson          3
alberti            2
BENITO JUAREZ      2
BAB                1
Name: count, dtype: int64

In [79]:
df['city'] = df['city'].replace({
    'Buenos Aires': 'Benito Juárez',
    'Henderson' : 'Benito Juárez',
    'BENITO JUAREZ' : 'Benito Juárez',
    'BAB' : 'Benito Juárez',
    'alberti' : 'Benito Juárez'
})
df['city'].value_counts()

city
Benito Juárez    704
Name: count, dtype: int64

In [80]:
# Se guarda el archivo principal de la ciudad
df.to_csv('../CleanData/JuarezMain.csv', index=False)

In [81]:
# Búsqueda de los archivos .csv en el directorio
glob.glob('../Datasets/*olavarria*.csv')

['../Datasets/olavarria_dataset_crawler-google-places_2025-06-15_23-35-08-815.csv',
 '../Datasets/olavarria_dataset_crawler-google-places_2025-07-30_14-43-37-977.csv',
 '../Datasets/olavarria_dataset_crawler-google-places_2025-05-29_00-56-36-905.csv',
 '../Datasets/olavarria_dataset_crawler-google-places_2025-06-11_18-14-26-674.csv']

In [82]:
# Carga y concatena todos los archivos CSV que contengan 'olavarria' en su nombre.
df = pd.concat(map(pd.read_csv, glob.glob('../Datasets/*olavarria*.csv'))).reset_index(drop=True)

In [83]:
# Selección de columnas relevantes
df = df[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df.head()

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Alto Salame Olavarria,Tienda de alimentación,NaN,Av. Domingo Faustino Sarmiento 2130,+54 2284 62-2206,Olavarría,Provincia de Buenos Aires,-36.881409,-60.317939,NaN,20,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T23:31:30.165Z,comercio
1,"Frutería y Verdulería ""El Nuevo Galpón""",Frutería,NaN,Av. de los Trabajadores 4312,+54 9 2284 35-5136,Olavarría,Provincia de Buenos Aires,-36.892194,-60.294718,4.5,19,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,15,2025-06-15T23:31:30.165Z,comercio
2,Despensa Bauti,Tienda de alimentación,NaN,Cerrito 4300,+54 9 2284 23-8406,Olavarría,Provincia de Buenos Aires,-36.879480,-60.314791,NaN,18,https://lh3.googleusercontent.com/gps-proxy/AL...,4.0,0,2025-06-15T23:31:30.165Z,comercio
3,Toxica,Tienda de ropa de mujer,NaN,Necochea 2757,NaN,Olavarría,Provincia de Buenos Aires,-36.891471,-60.320165,5.0,66,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,1,2025-06-15T23:31:33.614Z,comercio
4,"Kiosco ""El Quijote""",Quiosco,NaN,Lavalle 2880,NaN,Olavarría,Provincia de Buenos Aires,-36.891331,-60.324035,5.0,65,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,2,2025-06-15T23:31:33.614Z,comercio


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         2341 non-null   object 
 1   categoryName  2293 non-null   object 
 2   website       774 non-null    object 
 3   street        2253 non-null   object 
 4   phone         1797 non-null   object 
 5   city          2335 non-null   object 
 6   state         2335 non-null   object 
 7   location/lat  2341 non-null   float64
 8   location/lng  2341 non-null   float64
 9   totalScore    1867 non-null   float64
 10  rank          2341 non-null   int64  
 11  imageUrl      2289 non-null   object 
 12  imagesCount   2289 non-null   float64
 13  reviewsCount  2341 non-null   int64  
 14  scrapedAt     2341 non-null   object 
 15  searchString  2341 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 292.8+ KB


In [85]:
# Rellenar valores faltantes en la columna 'city'
df['city'] = df['city'].fillna('Olavarría')

<style>
.rendered_html {
    font-family: 'Comic Sans MS', cursive !important;
    font-size: 18px !important;
    color: red !important;
}
</style>

In [86]:
df['city'].value_counts()

city
Olavarría            2225
Gran Buenos Aires      57
KKJ                     4
DFE                     4
Buenos Aires            4
CVQ                     3
COO                     3
IDH                     3
B7400LCX                3
KCI                     3
DAS                     2
Caseros                 2
CZY                     2
DJY                     2
Moreno                  2
Colon                   2
JZB                     2
CUV                     1
B7400JWP                1
CWP                     1
CUR                     1
LNM                     1
DTO                     1
CABA                    1
JUE                     1
DRL                     1
DSN                     1
DJQ                     1
DRQ                     1
CUC                     1
CRB                     1
KKM                     1
DSC                     1
CUG                     1
LLD                     1
Name: count, dtype: int64

In [87]:
df['city'] = df['city'].replace({
    'Gran Buenos Aires': 'Olavarría',
    'DFE' : 'Olavarría',
    'Buenos Aires' : 'Olavarría',
    'KKJ' : 'Olavarría',
    'KCI' : 'Olavarría',
    'B7400LCX' : 'Olavarría',
    'COO' : 'Olavarría',
    'CVQ' : 'Olavarría',
    'IDH' : 'Olavarría',
    'Caseros' : 'Olavarría',
    'DAS' : 'Olavarría',
    'Colon' : 'Olavarría',
    'Moreno' : 'Olavarría',
    'CZY' : 'Olavarría',
    'DJY' : 'Olavarría',
    'JZB' : 'Olavarría',
    'DRQ' : 'Olavarría',
    'B7400JWP' : 'Olavarría',
    'CUV' : 'Olavarría',
    'CUG' : 'Olavarría',
    'DSC' : 'Olavarría',
    'KKM' : 'Olavarría',
    'CRB' : 'Olavarría',
    'CUC' : 'Olavarría',
    'JUE' : 'Olavarría',
    'DJQ' : 'Olavarría',
    'DSN' : 'Olavarría',
    'DRL' : 'Olavarría',
    'CWP' : 'Olavarría',
    'CUR' : 'Olavarría',
    'LNM' : 'Olavarría',
    'DTO' : 'Olavarría',
    'LLD' : 'Olavarría',
    'CABA' : 'Olavarría'
})

In [88]:
# Exporta el DataFrame a .CSV sin incluir la columna del índice
df.to_csv('../CleanData/OlavarriaMain.csv', index=False)

In [89]:

glob.glob('../Datasets/*rauch*.csv')

['../Datasets/rauch_dataset_google-maps-extractor_2025-05-19_00-06-21-606.csv',
 '../Datasets/rauch_dataset_crawler-google-places_2025-06-15_22-41-11-472.csv',
 '../Datasets/rauch_ dataset_crawler-google-places_2025-05-29_00-49-55-809.csv',
 '../Datasets/rauch_dataset_crawler-google-places_2025-07-30_15-11-01-826.csv']

In [90]:
df = pd.concat(map(pd.read_csv, glob.glob('../Datasets/*rauch*.csv'))).reset_index(drop=True)

In [91]:
df = df[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df.head()

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,"Despensa y anexo belen ""lo de jhonny""",Comercio,NaN,Barrio Flucho Casa 23,+54 9 249 448-5829,Rauch,Provincia de Buenos Aires,-36.780949,-59.078725,5.0,1,https://lh3.googleusercontent.com/p/AF1QipN4p-...,33.0,9,2025-05-19T00:05:29.686Z,Comercio
1,El Viejo Almacén,Comercio,NaN,NaN,+54 2297 44-2351,Rauch,Provincia de Buenos Aires,-36.777091,-59.088512,4.5,2,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,2,2025-05-19T00:05:29.688Z,Comercio
2,Campos Julian - Contenedores,Comercio,NaN,Av. Belgrano 573,+54 2297 44-2714,Rauch,Provincia de Buenos Aires,-36.779640,-59.081616,4.8,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,4,2025-05-19T00:05:29.688Z,Comercio
3,Arturo E Dualde,Comercio,NaN,Av. San Martín 1060,+54 2297 44-0384,Rauch,Provincia de Buenos Aires,-36.771673,-59.085032,4.5,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,2,2025-05-19T00:05:29.688Z,Comercio
4,Miguel A Siervo,Comercio,NaN,"ABB, 9 de Julio 62",+54 2297 44-0451,Rauch,Provincia de Buenos Aires,-36.776702,-59.092649,5.0,5,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,1,2025-05-19T00:05:29.691Z,Comercio


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         826 non-null    object 
 1   categoryName  806 non-null    object 
 2   website       147 non-null    object 
 3   street        746 non-null    object 
 4   phone         455 non-null    object 
 5   city          820 non-null    object 
 6   state         821 non-null    object 
 7   location/lat  826 non-null    float64
 8   location/lng  826 non-null    float64
 9   totalScore    527 non-null    float64
 10  rank          826 non-null    int64  
 11  imageUrl      802 non-null    object 
 12  imagesCount   802 non-null    float64
 13  reviewsCount  826 non-null    int64  
 14  scrapedAt     826 non-null    object 
 15  searchString  826 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 103.4+ KB


In [93]:
df['city'] = df['city'].fillna('Rauch')

In [94]:
df['city'].value_counts()

city
Rauch           722
BPQ              24
Buenos Aires      6
ADB               6
ACF               4
AEX               4
BBP               4
ASJ               4
ACD               4
BPR               4
BID               4
BPJ               4
AQH               4
BPO               3
ABD               3
BQC               3
ATQ               3
AQJ               3
AKG               3
BCB               2
BPP               2
BPK               2
BPI               2
AQE               1
BMT               1
AQF               1
AAI               1
BJF               1
AQB               1
Name: count, dtype: int64

In [95]:
df['city'] = df['city'].replace({
    'Buenos Aires': 'Rauch',
    'BPQ' : 'Rauch',
    'ADB' : 'Rauch',
    'ASJ' : 'Rauch',
    'BPR' : 'Rauch',
    'BBP' : 'Rauch',
    'ACD' : 'Rauch',
    'BPJ' : 'Rauch',
    'AEX' : 'Rauch',
    'BID' : 'Rauch',
    'ACF' : 'Rauch',
    'AQH' : 'Rauch',
    'BPO' : 'Rauch',
    'ATQ' : 'Rauch',
    'AQJ' : 'Rauch',
    'ABD' : 'Rauch',
    'BQC' : 'Rauch',    
    'AKG' : 'Rauch',
    'BCB' : 'Rauch',
    'BPP' : 'Rauch',
    'BPK' : 'Rauch',
    'BPI' : 'Rauch',
    'AQE' : 'Rauch',
    'BMT' : 'Rauch',
    'AQF' : 'Rauch',
    'AAI' : 'Rauch',
    'BJF' : 'Rauch',
    'AQB' : 'Rauch'
})

In [96]:
#Exportación del DataFrame a un archivo CSV
df.to_csv('../CleanData/RauchMain.csv', index=False)

In [97]:
glob.glob('../Datasets/*tandil*.csv')

['../Datasets/tandil_dataset_crawler-google-places_2025-06-11_18-03-13-154.csv',
 '../Datasets/tandil_dataset_crawler-google-places_2025-05-28_23-37-49-506.csv',
 '../Datasets/tandil_dataset_crawler-google-places_2025-05-29_00-18-14-608.csv',
 '../Datasets/tandil _dataset_google-maps-extractor_2025-05-04_16-09-01-070.csv',
 '../Datasets/tandil_dataset_crawler-google-places_2025-06-15_23-39-41-132.csv',
 '../Datasets/tandil_dataset_crawler-google-places_2025-07-30_15-45-22-219.csv',
 '../Datasets/tandil_dataset_crawler-google-places_2025-05-28_23-44-51-919.csv']

In [98]:
df = pd.concat(map(pd.read_csv, glob.glob('../Datasets/*tandil*.csv'))).reset_index(drop=True)


In [99]:
df = df[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df.head()

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Marcovecchio Bikes,Tienda de bicicletas,https://marcovecchiobikes.com/,"Astor Piazzola 160 Entre Ijurco y, Cesar Milstein",+54 249 401-9991,Tandil,Provincia de Buenos Aires,-37.282066,-59.102040,4.9,1,https://lh3.googleusercontent.com/p/AF1QipNLaf...,76.0,371,2025-06-11T18:01:49.484Z,venta online
1,El Ultimo Recurso,Comercio,NaN,Ijurco 2243,NaN,Tandil,Provincia de Buenos Aires,-37.294901,-59.115855,5.0,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,1,2025-06-11T18:01:50.668Z,comercio
2,Enever tandil,Comercio,https://www.instagram.com/enever_tandil?igsh=a...,"pasaje interno 1850 ENTRE LA CALLE IJURCO, Juá...",+54 249 428-7471,Tandil,Provincia de Buenos Aires,-37.298936,-59.120144,5.0,2,https://lh3.googleusercontent.com/p/AF1QipP2Tl...,6.0,1,2025-06-11T18:01:50.668Z,comercio
3,El Surtidor Tandil,Comercio,https://sites.google.com/view/el-surtidor-tand...,"Darragueira, Ijurco 1999",+54 249 462-2606,Tandil,Provincia de Buenos Aires,-37.300845,-59.123275,4.4,3,https://lh3.googleusercontent.com/p/AF1QipNqjA...,18.0,280,2025-06-11T18:01:50.668Z,comercio
4,A limpiar,Comercio,NaN,Ijurco 1883,+54 249 464-4277,Tandil,Provincia de Buenos Aires,-37.298287,-59.120049,4.3,4,https://lh3.googleusercontent.com/p/AF1QipMcYB...,39.0,47,2025-06-11T18:01:50.668Z,comercio


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         3577 non-null   object 
 1   categoryName  3498 non-null   object 
 2   website       1360 non-null   object 
 3   street        3450 non-null   object 
 4   phone         2746 non-null   object 
 5   city          3543 non-null   object 
 6   state         3555 non-null   object 
 7   location/lat  3577 non-null   float64
 8   location/lng  3577 non-null   float64
 9   totalScore    2966 non-null   float64
 10  rank          3577 non-null   int64  
 11  imageUrl      3486 non-null   object 
 12  imagesCount   3485 non-null   float64
 13  reviewsCount  3577 non-null   int64  
 14  scrapedAt     3577 non-null   object 
 15  searchString  3577 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 447.3+ KB


In [101]:
df['city'] = df['city'].fillna('Tandil')

In [102]:
df['city'].value_counts()

city
Tandil                             3486
Gran Buenos Aires                    21
Buenos Aires                         11
FTE                                   4
HXG                                   4
IRK                                   4
BGA                                   4
GRS                                   3
GZY                                   3
GKP                                   3
AOX                                   2
AOS                                   2
FSW                                   2
AQX                                   2
ATN                                   2
IRL                                   2
BKB                                   2
AZR                                   1
AHM                                   1
AZG                                   1
BWH                                   1
GMM                                   1
AZC                                   1
AZD                                   1
AOV                                

In [103]:
df['city'] = df['city'].replace({
    'Gran Buenos Aires': 'Tandil',
    'Buenos Aires': 'Tandil',
    'HXG': 'Tandil',
    'IRK': 'Tandil',
    'BGA': 'Tandil',
    'FTE': 'Tandil',
    'GRS': 'Tandil',
    'GZY': 'Tandil',
    'GKP': 'Tandil',
    'ATN': 'Tandil',
    'BWH': 'Tandil',
    'BKB': 'Tandil',
    'AOS': 'Tandil',
    'AQX': 'Tandil',
    'AOX': 'Tandil',
    'FSW': 'Tandil',
    'IRL': 'Tandil',
    'AHM': 'Tandil',
    'AQO': 'Tandil',
    'HEU': 'Tandil',
    'BJD': 'Tandil',
    'Gutiérrez': 'Tandil',
    'AZR': 'Tandil',
    'AZG': 'Tandil',
    'EAC': 'Tandil',
    'GKR': 'Tandil',
    'GQQ': 'Tandil',
    'AOP': 'Tandil',
    'AOO': 'Tandil',
    'AEK': 'Tandil',
    'GVL': 'Tandil',
    'GMM': 'Tandil',
    'AZC': 'Tandil',
    'AZD': 'Tandil',
    'AOV': 'Tandil',
    'Ciudad Autónoma de Buenos Aires': 'Tandil'
})

In [104]:
#Exportación del DataFrame a un archivo CSV
df.to_csv('../CleanData/TandilMain.csv', index=False)

In [105]:
# Búsqueda de los archivos .csv en el directorio
glob.glob('../CleanData/*Main*.csv')

['../CleanData/AzulMain.csv',
 '../CleanData/OlavarriaMain.csv',
 '../CleanData/JuarezMain.csv',
 '../CleanData/TandilMain.csv',
 '../CleanData/RauchMain.csv']

In [106]:
# Lectura y unión de archivos CSV
df = pd.concat(map(pd.read_csv, glob.glob('../CleanData/*Main*.csv'))).reset_index(drop=True)

In [107]:
# Información del DataFrame a través del método .info() : 
# Número de filas y columnas
# Nombres y tipos de datos de las columnas
# Valores no nulos en cada columna
# Uso de memoria
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9178 entries, 0 to 9177
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         9178 non-null   object 
 1   categoryName  8982 non-null   object 
 2   website       2783 non-null   object 
 3   street        8688 non-null   object 
 4   phone         6535 non-null   object 
 5   city          9178 non-null   object 
 6   state         9129 non-null   object 
 7   location/lat  9178 non-null   float64
 8   location/lng  9178 non-null   float64
 9   totalScore    7089 non-null   float64
 10  rank          9178 non-null   int64  
 11  imageUrl      8936 non-null   object 
 12  imagesCount   8933 non-null   float64
 13  reviewsCount  9178 non-null   int64  
 14  scrapedAt     9178 non-null   object 
 15  searchString  9178 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 1.1+ MB


In [108]:
# Se verifica si hay valores nulos en la columna 'state'
df['state'] = df['state'].fillna('Provincia de Buenos Aires')

In [109]:
# Este código cuenta la frecuencia de cada registro en la columna 'state'
df['state'].value_counts()

state
Provincia de Buenos Aires          9163
Ciudad Autónoma de Buenos Aires       6
Buenos Aires Sur                      4
de Buenos Aires                       2
San Juan                              1
La Pampa                              1
CABA                                  1
Name: count, dtype: int64

In [110]:
# Se reemplazan los datos de la columna 'state' que no son correctos
df['state'] = df['state'].replace({
    'San Juan': 'Provincia de Buenos Aires',
    'La Pampa': 'Provincia de Buenos Aires',
    'Ciudad Autónoma de Buenos Aires': 'Provincia de Buenos Aires',
    'de Buenos Aires': 'Provincia de Buenos Aires',
    'Ciudad Autónoma de Buenos Aires': 'Provincia de Buenos Aires',
    'CABA': 'Provincia de Buenos Aires',
    'Buenos Aires Sur' : 'Provincia de Buenos Aires' 
})


In [111]:
# Define qué strings deben ser tratados como nulos en las columnas seleccionadas.
nulls = ['', ' ', 'N/A', 'nan', 'NaN', 'None']
# Define las columnas a las que quieres aplicar este proceso.
col = ['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state', 'imageUrl']
# Itera sobre cada columna y aplica la lógica.
for col in col:
    df.loc[:, col] = df[col].replace(nulls, np.nan) # Estandariza a np.nan
    df.loc[:, col] = df[col].fillna('Incomplete') # Reemplaza nulos con 'Incomplete'


In [112]:
# Define las columnas a las que se aplica este proceso.
colnum = ['location/lat' ,'location/lng', 'totalScore', 'imagesCount', 'rank', 'reviewsCount']
# Itera sobre cada columna y aplica la lógica.
for colnum in colnum:
    df.loc[:, colnum] = df[colnum].replace(nulls, np.nan) # Estandariza a np.nan
    df.loc[:, colnum] = df[colnum].fillna(0) # Reemplaza nulos con 0

In [113]:
# Verifica si hay valores nulos en el DataFrame.
df.isna().sum()

title           0
categoryName    0
website         0
street          0
phone           0
city            0
state           0
location/lat    0
location/lng    0
totalScore      0
rank            0
imageUrl        0
imagesCount     0
reviewsCount    0
scrapedAt       0
searchString    0
dtype: int64

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9178 entries, 0 to 9177
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         9178 non-null   object 
 1   categoryName  9178 non-null   object 
 2   website       9178 non-null   object 
 3   street        9178 non-null   object 
 4   phone         9178 non-null   object 
 5   city          9178 non-null   object 
 6   state         9178 non-null   object 
 7   location/lat  9178 non-null   float64
 8   location/lng  9178 non-null   float64
 9   totalScore    9178 non-null   float64
 10  rank          9178 non-null   int64  
 11  imageUrl      9178 non-null   object 
 12  imagesCount   9178 non-null   float64
 13  reviewsCount  9178 non-null   int64  
 14  scrapedAt     9178 non-null   object 
 15  searchString  9178 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 1.1+ MB


In [115]:
# Detecta y cuenta filas duplicadas basadas en título, ciudad, calle y coordenadas
dup = df.duplicated(subset=['title','city','street','location/lat','location/lng']).sum()
dup.sum()
print(f"Hay {dup} filas duplicadas en el DataFrame.")

Hay 3319 filas duplicadas en el DataFrame.


In [116]:
# Quita duplicados según campos clave (título, ciudad, calle y coordenadas), reinicia el índice y 
# guarda una copia limpia del DataFrame.
df = df.loc[~df.duplicated(subset=['title','city','street','location/lat','location/lng'])] \
    .reset_index(drop=True).copy()

In [117]:
# Verifica estructura, cuenta de filas y columnas, tipos y valores faltantes
# tras limpieza de duplicados y modificación de tipo de dato
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5859 entries, 0 to 5858
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         5859 non-null   object 
 1   categoryName  5859 non-null   object 
 2   website       5859 non-null   object 
 3   street        5859 non-null   object 
 4   phone         5859 non-null   object 
 5   city          5859 non-null   object 
 6   state         5859 non-null   object 
 7   location/lat  5859 non-null   float64
 8   location/lng  5859 non-null   float64
 9   totalScore    5859 non-null   float64
 10  rank          5859 non-null   int64  
 11  imageUrl      5859 non-null   object 
 12  imagesCount   5859 non-null   float64
 13  reviewsCount  5859 non-null   int64  
 14  scrapedAt     5859 non-null   object 
 15  searchString  5859 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 732.5+ KB


In [118]:
# Limpia y estandariza números internacionales (+54) removiendo dígito “9” extra antes del código local
df['phone'] = df['phone'].astype(str).str.replace(r'\+54\s*9\s*(\d+)', r'+54 \1', regex=True)
df['phone']

0             Incomplete
1       +54 2281 50-2996
2       +54 2281 31-0611
3       +54 2281 42-3119
4       +54 2281 43-0338
              ...       
5854    +54 2297 44-2572
5855    +54 249 469-4342
5856    +54 2297 44-0468
5857    +54 249 435-9904
5858    +54 249 421-6088
Name: phone, Length: 5859, dtype: object

In [119]:
# Estándariza números telefónicos agregando ‘9’ después del +54 si falta, asegurando el formato internacional correcto.
df['phone'] = df['phone'].str.replace(
    r'^\+54\s*(?!9\s*[\d(])(.+)$', 
    r'+54 9 \1',                   
    regex=True
)
df['phone']

0               Incomplete
1       +54 9 2281 50-2996
2       +54 9 2281 31-0611
3       +54 9 2281 42-3119
4       +54 9 2281 43-0338
               ...        
5854    +54 9 2297 44-2572
5855    +54 9 249 469-4342
5856    +54 9 2297 44-0468
5857    +54 9 249 435-9904
5858    +54 9 249 421-6088
Name: phone, Length: 5859, dtype: object

In [120]:
# Limpia los nombres de comercios eliminando comillas, asteriscos, signos de interrogación y emoticones.
df['title'] = df['title'].str.replace('"', '', regex=False) \
    .str.replace('*', '', regex=False) \
    .str.replace('?', '', regex=False) \
    .str.replace("'", '', regex=False) \
    .str.replace("👬", '', regex=False) \
    .str.replace("✨", '', regex=False) \
    .str.replace("⚽️", '', regex=False) \
    .str.replace("💖", '', regex=False) \
    .str.replace("♓", '', regex=False) \
    .str.replace("°", '', regex=False) \
    .str.replace("❣️", '', regex=False) \
    .str.replace("!", '', regex=False) \
    .str.replace("🔥", '', regex=False) \
    .str.replace("🪽", '', regex=False)	  
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,La Casa de los Trofeos,Comercio,Incomplete,Rivadavia 496,Incomplete,Azul,Provincia de Buenos Aires,-36.774661,-59.862623,0.0,1,Incomplete,0.0,0,2025-05-19T00:03:00.547Z,Comercio
1,La Escoba Loca,Tienda de artículos para el hogar,Incomplete,Colón Nte. 117,+54 9 2281 50-2996,Azul,Provincia de Buenos Aires,-36.767281,-59.851848,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,44.0,73,2025-05-19T00:03:02.157Z,Comercio
2,Artículos magali,Tienda general,Incomplete,Monseñor Caneva Nte. 160,+54 9 2281 31-0611,Azul,Provincia de Buenos Aires,-36.774353,-59.841898,4.3,2,Incomplete,0.0,12,2025-05-19T00:03:02.157Z,Comercio
3,Autoservicio San Jorge,Comercio,Incomplete,Burgos Nte. 100,+54 9 2281 42-3119,Azul,Provincia de Buenos Aires,-36.768145,-59.851305,4.6,3,https://lh3.googleusercontent.com/p/AF1QipPV0V...,6.0,127,2025-05-19T00:03:02.157Z,Comercio
4,Asociacion Latina de Exportacion SA,Comercio,Incomplete,Julio Argentino Roca 846,+54 9 2281 43-0338,Azul,Provincia de Buenos Aires,-36.785067,-59.859254,3.3,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,4,2025-05-19T00:03:02.158Z,Comercio


In [121]:
# Estandariza title, searchString y categoryName aplicando formato de título.
df['title'] = df['title'].str.title()
df['searchString'] = df['searchString'].str.title()
df['categoryName'] = df['categoryName'].str.title()
df[['title', 'searchString', 'categoryName']]

,title,searchString,categoryName
0,La Casa De Los Trofeos,Comercio,Comercio
1,La Escoba Loca,Comercio,Tienda De Artículos Para El Hogar
2,Artículos Magali,Comercio,Tienda General
3,Autoservicio San Jorge,Comercio,Comercio
4,Asociacion Latina De Exportacion Sa,Comercio,Comercio
...,...,...,...
5854,Cerrajeria Hugo,Construccion,Cerrajero
5855,Signos Arte Grafico,Construccion,Servicios De Diseño
5856,Registro Nacional De La Propiedad Del Automotor,Vehiculos,Agencia De Registro De Automóviles
5857,Carpintería Tedesco Hermanos,Construccion,Carpintero


In [122]:
# Elimina el prefijo '/url?q=' de las URLs en la columna 'website' y conserva solo la URL base.
df['website'] = df['website'].str.replace(
    r'^\s*/url\?q=(https?://.*)',
    r'\1',
    regex=True
)

In [123]:
# Contar cuántas veces aparece cada valor en la columna 'website'
df['website'].value_counts()

website
Incomplete                                     4109
https://argentina.gridohelado.com/                6
http://www.cooperativaobrera.coop/                5
http://www.luz-azul.com.ar/                       5
http://www.ypf.com/                               4
                                               ... 
http://www.zucchiattipropiedades.com.ar/          1
https://www.millenniumpropiedades.com/            1
http://www.alvarezneumaticos.com/                 1
http://www.plastigas.com.ar/                      1
https://www.instagram.com/benditoolavarria/       1
Name: count, Length: 1657, dtype: int64

In [124]:
# Se cuentan las distintas categorías
category = df['categoryName'].nunique()
print(f"Los comercios están registrados en un total de {category} categorías distintas")

Los comercios están registrados en un total de 621 categorías distintas


In [125]:
# Se cuentan las categorías y se muestran las más frecuentes.
df['categoryName'].value_counts()

categoryName
Tienda De Ropa                   464
Tienda De Alimentación           366
Comercio                         333
Agencia Inmobiliaria             180
Incomplete                       142
                                ... 
Escuela De Artes Marciales         1
Tienda De Cristales Y Espejos      1
Complejo De Viviendas              1
Cerrajero                          1
Zapatero                           1
Name: count, Length: 621, dtype: int64

In [126]:
# Se exporta el DataFrame Regional limpio a un archivo CSV sin incluir la columna del índice.
df.to_csv('../CleanData/Regional.csv', index=False)

<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">
✅ Conclusión

Se obtuvo un dataset unificado, sin duplicados y con valores nulos tratados.
El archivo está limpio y preparado para el análisis exploratorio.

</div>